# Compare Chains (compatible with inviz < 0.0.6)
Let's say that you suspect that some of your chains are corrupted, and you want to see where they live in the parameter space. Here's how inviz can help you do that!

In [ ]:
from inviz import *
hv.extension('bokeh')
pn.extension()

In [25]:
from inviz import cosmo
import holoviews as hv
from holoviews import dim, opts, streams
import pandas as pd
import numpy as np
from tqdm import trange, tqdm
import panel as pn
import spatialpandas
from bokeh.models import HoverTool


First we need to load the chains, which are plain txt files, and process them into Holoviews Dataset objects. Inviz has two functions that can process your chains:
- `load_params` : parses the *.paramnames* file
- `load_data` : parses the *.txt* files in which the chains are located

Suppose the last 6 chains in the `test_IDM_n_0` dataset are bad. We can slit them up into two dataframes like so:

In [28]:
param_names, latex = cosmo.load_params('../data/test_IDM_n_0/2022-05-04_75000_.paramnames')
param_names = ['loglkl'] + param_names
good_df = pd.DataFrame(columns=param_names)
bad_df = pd.DataFrame(columns=param_names)

for i in trange(1,51):
    temp1 = cosmo.load_data('../data/test_IDM_n_0/2022-05-04_75000__{}.txt'.format(i), column_names=param_names)
    good_df = pd.concat([good_df,temp1]).reset_index(drop=True)
    
for j in trange(51,57):
    temp2 = cosmo.load_data('../data/test_IDM_n_0/2022-05-04_75000__{}.txt'.format(j), column_names=param_names)
    bad_df = pd.concat([bad_df,temp2]).reset_index(drop=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 17.84it/s]


If you want to see less data points in the final scatter plot, you can slice the dataframes. After slicing, we turn them into Holoviews Dataset objects. These objects allow us to specify **labels** for our data and **key dimensions** (kdims) which will become our x and y axes.

In [29]:
good_df_slice = good_df[::200]
new_good_df = good_df_slice.reset_index(drop=True)
bad_df_slice = bad_df[::200]
new_bad_df = bad_df_slice.reset_index(drop=True)

# modify the kdims to be any two parameters you want. make sure they match so they can be overlaid on the same plot.
good_ds = hv.Dataset(new_good_df, ['omega_b', '100theta_s'], label='good')
bad_ds = hv.Dataset(new_bad_df, ['omega_b', '100theta_s'], label='bad')

Those datasets will then be passed into the following function (it's not part of the official package on pyPI yet, which is why it's written out here):

In [30]:
def compare_two_chainz(dataset1, dataset2):
    hover = HoverTool(tooltips=None)
    
    # generate point plots for both datasets and set them as data stream sources
    pts1 = hv.Points(dataset1)
    pts2 = hv.Points(dataset2)
    sel1 = streams.Selection1D(source=pts1)
    sel2 = streams.Selection1D(source=pts2)

    colorcode_good_bad = hv.Overlay([pts1, pts2]).opts(
        opts.Points(
            width=400,
            height=400,
            size=5,
            alpha=0.5,
            selection_alpha=1,
            nonselection_alpha=0.1,
            tools=[hover,'box_select','lasso_select','tap']
        ),
        opts.Overlay(
            legend_position='bottom_right'
        )
    )

    # generate a table of all the selected points on the point plot
    def sel_table(dataset, stream):
        table = hv.DynamicMap(lambda index: hv.Table(dataset.iloc[index]), streams=[stream])
        return table

    # do it for both datasets
    good_table = sel_table(dataset1, sel1)
    bad_table = sel_table(dataset2, sel2)

    # formatting the table
    def hook(plot, element):
        plot.handles['table'].autosize_mode = "none"
        for column in plot.handles['table'].columns:
            column.width = 100

    table_options = opts.Table(
        height=150,
        width=1000,
        hooks=[hook]
    )

    # put it all together
    selections_good_bad = good_table + bad_table
    selections_good_bad = selections_good_bad.opts(table_options).cols(1)
    dashboard = pn.Row(colorcode_good_bad, selections_good_bad)
    return dashboard

Call the function with our good and bad datasets as arguments:

In [31]:
compare_two_chainz(good_ds, bad_ds)

Row
    [0] HoloViews(Overlay, height=400, sizing_mode='fixed', width=400)
    [1] HoloViews(Layout)